<a href="https://colab.research.google.com/github/mayanksonu11/Tdoc-Downloader/blob/main/TDoc_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from openpyxl import load_workbook
import datetime
import os
import requests
from tqdm import tqdm
import zipfile
import shutil

# New Section

In [ ]:
# search the xlsx file in the cur rent directory and load them
xlsx_files = [f for f in os.listdir() if f.endswith('.xlsx')]
if not xlsx_files:
    print("No .xlsx files found in the current directory.")
else:
    # Load the first .xlsx file
    xlsx_file = xlsx_files[0]
    print(f"Loading {xlsx_file}...")
    wb = load_workbook(xlsx_file)
    # Check if the file is loaded successfully
    if wb:
        print(f"Loaded {xlsx_file} successfully.")
    else:
        print(f"Failed to load {xlsx_file}.")

# Select the sheet
sheet = wb['TDoc_List']
agenda_item = "8.5.4"

Loading TDoc_List_Meeting_RAN2#128.xlsx...
Loaded TDoc_List_Meeting_RAN2#128.xlsx successfully.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Extract hyperlinks from the TDoc column based on conditions
tdoc_links = {}
for row in sheet.iter_rows(min_row=2, max_row=1500):  # Adjust min_row as needed
    tdoc_cell = row[0]  # Assuming TDoc column is the first column
    agenda_item_cell = row[10]  # Assuming Agenda item is the second column
    status_cell = row[13]  # Assuming TDoc Status is the third column
    title = row[1] # title cell
    # print(row)
    # print(status_cell.value)
    # Check conditions for Agenda item and TDoc Status
    if (
        # agenda_item_cell.value == agenda_item and
        "CU" in title.value and
        status_cell.value == "available"
    ):
        tdoc_links[tdoc_cell.value] = tdoc_cell.hyperlink.target

print(tdoc_links)

{'R2-2409616': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2409616.zip', 'R2-2409863': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2409863.zip', 'R2-2409873': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2409873.zip', 'R2-2409886': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2409886.zip', 'R2-2409888': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2409888.zip', 'R2-2409953': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2409953.zip', 'R2-2409988': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2409988.zip', 'R2-2410012': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2410012.zip', 'R2-2410021': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2410021.zip', 'R2-2410035': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2410035.zip', 'R2-2410044': 'https://www.3gpp.org/ftp/TSG_RAN/WG2_RL2/TSGR2_128/Docs/R2-2410044.zip', 'R2-2410064': 'https://www.3gpp

In [ ]:
# iterate over all the links and download it into a folder

# Create a directory to save the downloaded files
download_dir = "TDoc_Downloads"
os.makedirs(download_dir, exist_ok=True)
# Download each file
for tdoc_name, link in tdoc_links.items():
    # Extract the filename from the link
    filename = os.path.join(download_dir, os.path.basename(link))
    # Check if the file already exists
    if os.path.exists(filename):
        print(f"File {filename} already exists. Skipping download.")
        continue
    # Download the file
    print(f"Downloading {tdoc_name} from {link}...")
    response = requests.get(link, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    with open(filename, 'wb') as f:
        for data in tqdm(response.iter_content(chunk_size=1024), total=total_size // 1024, unit='KB'):
            f.write(data)
    print(f"Downloaded {tdoc_name} to {filename}.")
# Print the downloaded files
print("Downloaded files:")
for filename in os.listdir(download_dir):
    print(filename)


File TDoc_Downloads/R2-2409616.zip already exists. Skipping download.
File TDoc_Downloads/R2-2409863.zip already exists. Skipping download.
File TDoc_Downloads/R2-2409873.zip already exists. Skipping download.
File TDoc_Downloads/R2-2409886.zip already exists. Skipping download.
File TDoc_Downloads/R2-2409888.zip already exists. Skipping download.
File TDoc_Downloads/R2-2409953.zip already exists. Skipping download.
File TDoc_Downloads/R2-2409988.zip already exists. Skipping download.
File TDoc_Downloads/R2-2410012.zip already exists. Skipping download.
File TDoc_Downloads/R2-2410021.zip already exists. Skipping download.
File TDoc_Downloads/R2-2410035.zip already exists. Skipping download.
File TDoc_Downloads/R2-2410044.zip already exists. Skipping download.
File TDoc_Downloads/R2-2410064.zip already exists. Skipping download.
File TDoc_Downloads/R2-2410113.zip already exists. Skipping download.
File TDoc_Downloads/R2-2410115.zip already exists. Skipping download.
File TDoc_Downloads/

In [ ]:
# iterate over the downloaded files and extract them into separate folders as per the TDoc name
# Create a directory to save the extracted files
extract_dir = "TDoc_Extracts"
os.makedirs(extract_dir, exist_ok=True)
# Extract each file
for tdoc_name, link in tdoc_links.items():
    # Extract the filename from the link
    filename = os.path.join(download_dir, os.path.basename(link))
    # Create a directory for the extracted files
    extract_subdir = os.path.join(extract_dir, tdoc_name)
    os.makedirs(extract_subdir, exist_ok=True)
    # Check if the file is a zip file
    if filename.endswith('.zip'):
        print(f"Extracting {filename} to {extract_subdir}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(extract_subdir)
        print(f"Extracted {filename} to {extract_subdir}.")
    else:
        print(f"{filename} is not a zip file. Skipping extraction.")
# Print the extracted files
print("Extracted files:")
for tdoc_name in os.listdir(extract_dir):
    print(f"Files extracted for {tdoc_name}:")
    for filename in os.listdir(os.path.join(extract_dir, tdoc_name)):
        print(filename)

Extracting TDoc_Downloads/R2-2409616.zip to TDoc_Extracts/R2-2409616...
Extracted TDoc_Downloads/R2-2409616.zip to TDoc_Extracts/R2-2409616.
Extracting TDoc_Downloads/R2-2409863.zip to TDoc_Extracts/R2-2409863...
Extracted TDoc_Downloads/R2-2409863.zip to TDoc_Extracts/R2-2409863.
Extracting TDoc_Downloads/R2-2409873.zip to TDoc_Extracts/R2-2409873...
Extracted TDoc_Downloads/R2-2409873.zip to TDoc_Extracts/R2-2409873.
Extracting TDoc_Downloads/R2-2409886.zip to TDoc_Extracts/R2-2409886...
Extracted TDoc_Downloads/R2-2409886.zip to TDoc_Extracts/R2-2409886.
Extracting TDoc_Downloads/R2-2409888.zip to TDoc_Extracts/R2-2409888...
Extracted TDoc_Downloads/R2-2409888.zip to TDoc_Extracts/R2-2409888.
Extracting TDoc_Downloads/R2-2409953.zip to TDoc_Extracts/R2-2409953...
Extracted TDoc_Downloads/R2-2409953.zip to TDoc_Extracts/R2-2409953.
Extracting TDoc_Downloads/R2-2409988.zip to TDoc_Extracts/R2-2409988...
Extracted TDoc_Downloads/R2-2409988.zip to TDoc_Extracts/R2-2409988.
Extracting TD

In [ ]:
%pip install python-docx transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.4 MB/s eta 0:00:00


In [ ]:
import docx
from transformers import pipeline

def read_docx(file_path):
    """Reads text from a .docx file.

    Args:
        file_path: The path to the .docx file.

    Returns:
        A string containing the text content of the document.
    """
    doc = docx.Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return "\n".join(text)

def summarize_text(text):
    """Summarizes the given text using a pre-trained model.

    Args:
        text: The input text to summarize.

    Returns:
        A string containing the summarized text.
    """
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    # The summarizer has a maximum input size, so we might need to split
    # the text if it's too long. For this example, we'll just truncate.
    max_input_length = 1024  # Adjust this based on the model
    summarized_text = summarizer(text[:max_input_length], max_length=1000, min_length=30, do_sample=False)
    return summarized_text[0]['summary_text']

In [ ]:
import csv
import os

# Create a list to store the summaries
summaries = []

# Iterate over the extracted files
for tdoc_name in os.listdir(extract_dir):
    tdoc_path = os.path.join(extract_dir, tdoc_name)
    if os.path.isdir(tdoc_path):
        for filename in os.listdir(tdoc_path):
            if filename.endswith('.docx') and not filename.startswith('__MACOSX'):
                file_path = os.path.join(tdoc_path, filename)
                try:
                    document_text = read_docx(file_path)
                    print(document_text)
                    summary = summarize_text(document_text)
                    summaries.append({'TDoc Name': tdoc_name, 'Filename': filename, 'Summary': summary})
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Define the output CSV file path
csv_file_path = "tdoc_summaries.csv"

# Write the summaries to a CSV file
if summaries:
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['TDoc Name', 'Filename', 'Summary'])
        writer.writeheader()
        writer.writerows(summaries)
    print(f"Summaries saved to {csv_file_path}")
else:
    print("No .docx files found to summarize.")

3GPP TSG-RAN WG2 #128                                             	    R2-2410530                                                              
Orlando, FL, USA, November 18 – 22 , 2024                                                 	     

Agenda item:	   8.6.2
Source: 		Qualcomm Incorporated
Title: 	Security handling for Inter-CU LTM
Document for:	Discussion
1	Introduction
RAN2 received a response LS (S3-244316) from SA3 on the questions posed by RAN2 on the security handling for inter-CU LTM.
SA WG3 has been considering the security mechanisms proposed for the inter-CU LTM feature. For this feature, SA WG3 want to preserve the principle that when there is a PDCP anchor change, then it is necessary to perform a key change (i.e. for solutions or handovers that do not require a PDCP anchor change, there is no need to provide rekeying).
The rekeying method preferred by most companies in SA WG3 is that after every inter-CU LTM cell switch execution that changes the PDCP anchor, the UE i

Device set to use cpu
Your max_length is set to 1000, but your input_length is only 412. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=206)


KeyboardInterrupt: 